# 🏦 Multi-Agent Workflow: Insurance Claims Processing

This notebook demonstrates how to build a **multi-agent workflow** using Azure AI Agent Service's `WorkflowAgentDefinition` with **YAML-based workflow orchestration** for automated insurance claims processing.

## What You'll Learn

- How to create specialist agents with specific roles
- How to define a **workflow YAML** that orchestrates multiple agents
- How to use `WorkflowAgentDefinition` for declarative multi-agent coordination
- How to process streaming workflow events

## Architecture Overview

The system uses a **declarative YAML workflow** that coordinates:
1. **Validity Agent**: Assesses claim validity
2. **Department Agent**: Assigns the appropriate department
3. **Payout Agent**: Estimates payout range
4. **Orchestrator Agent**: Synthesizes all assessments

### Workflow Pattern (YAML-based)

```yaml
kind: workflow
trigger: OnConversationStart
actions:
  - InvokeAzureAgent (validity)
  - InvokeAzureAgent (department)  
  - InvokeAzureAgent (payout)
  - InvokeAzureAgent (orchestrator)
```



## 📦 Import Required Libraries and Setup Environment

This cell imports all the necessary libraries for building our multi-agent insurance claims processing system and loads environment variables from the `.env` file. We need:

- **Azure AI Agents SDK**: To create and manage multiple AI agents
- **Azure Identity**: For authentication with Azure services
- **Environment variables**: Project endpoint and model deployment details

In [ ]:
import os
from pathlib import Path
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    WorkflowAgentDefinition,
    ResponseStreamEventType,
    ItemType,
)
from azure.identity import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Get tenant ID and project endpoint
tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"🔑 Using Tenant ID: {tenant_id}")
print(f"🔗 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")

# Verify we have all required environment variables
if not project_endpoint:
    print("❌ Error: AI_FOUNDRY_PROJECT_ENDPOINT not found in environment variables")
    print("💡 Make sure your .env file contains AI_FOUNDRY_PROJECT_ENDPOINT")
elif not model_deployment:
    print("❌ Error: AZURE_AI_MODEL_DEPLOYMENT_NAME not found in environment variables") 
    print("💡 Make sure your .env file contains AZURE_AI_MODEL_DEPLOYMENT_NAME")
else:
    print("✅ All required environment variables found")

## 🎯 Define Specialist Agent Instructions

Now we'll define the instructions for each of our four specialist agents. Each agent has a specific role in the insurance claims processing workflow.

### Claim Validity Agent

This agent analyzes claims to determine their validity based on policy coverage, documentation, and claim details.

In [ ]:
# Claim Validity Agent definition
validity_agent_name = "claim-validity-agent"
validity_agent_instructions = """
Assess whether an insurance claim is valid based on its description and coverage details.

Respond with one of the following statuses:
- Valid: Claim is covered under policy and documentation is complete
- Requires Review: Additional documentation or investigation needed
- Denied: Claim is not covered or policy exclusions apply

Only output the validity status and a very brief explanation of your determination.
"""

### Department Assignment Agent

This agent determines which claims department should handle each claim based on the type of insurance and nature of the claim.

In [ ]:
# Department Assignment Agent definition
department_agent_name = "department-assignment-agent"
department_agent_instructions = """
Decide which claims department should handle each insurance claim.

Choose from the following departments:
- Auto Claims: Vehicle accidents, theft, damage
- Home Claims: Property damage, theft, liability
- Life Claims: Death benefits, policy payouts
- Health Claims: Medical expenses, hospitalization
- Commercial Claims: Business-related insurance claims

Base your answer on the type of incident described. Respond with the department name and a very brief explanation.
"""

### Payout Estimation Agent

This agent estimates the potential payout amount based on the claim details, policy limits, and deductibles.

In [ ]:
# Payout Estimation Agent definition
payout_agent_name = "payout-estimation-agent"
payout_agent_instructions = """
Estimate the potential payout range for each insurance claim.

Use the following scale:
- Low: Under $5,000 - minor repairs, small medical expenses
- Medium: $5,000-$25,000 - significant repairs, moderate medical treatment
- High: Over $25,000 - major damage, extensive medical care, total loss

Base your estimate on the severity of the incident described. Respond with the payout level and a brief justification.
"""

### Claims Orchestrator Agent Instructions

The orchestrator agent coordinates all specialist agent outputs and provides a comprehensive claims processing recommendation.

In [ ]:
# Instructions for the orchestrator claims processing agent
orchestrator_agent_name = "claims-orchestrator-agent"
orchestrator_agent_instructions = """
You are a senior claims analyst who synthesizes assessments from multiple specialist agents.

When you receive an insurance claim, review all the context from the conversation which includes:
1. The original claim details
2. Validity assessment from the validity specialist
3. Department assignment from the department specialist
4. Payout estimation from the payout specialist

Your job is to:
1. Review the original claim and all specialist assessments in the conversation
2. Identify any inconsistencies or concerns
3. Provide a final comprehensive claims processing recommendation
4. Include next steps for the claims adjuster

Format your response as a structured claims report with clear sections:
- CLAIM SUMMARY
- VALIDITY STATUS
- ASSIGNED DEPARTMENT
- PAYOUT ESTIMATE
- FINAL RECOMMENDATION
- NEXT STEPS
"""

## 🔗 Connect to Azure AI Agent Service

This cell establishes a connection to the Azure AI Agent Service using our project endpoint and credentials. This client will be used to create and manage all our insurance claims processing agents.

In [ ]:
# Connect to the AIProjectClient using AzureCliCredential
credential = AzureCliCredential()
print("🔐 Using AzureCliCredential for authentication...")

# Initialize the project client
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

# Get OpenAI client for conversations and responses
openai_client = project_client.get_openai_client()

print("✅ AIProjectClient initialized with AzureCliCredential")
print("✅ OpenAI client ready for conversations")

## 🤖 Create Multi-Agent Insurance Claims System

This cell creates all agents in our multi-agent system:

1. **Three Specialist Agents**: Validity, Department, and Payout assessment agents
2. **One Orchestrator Agent**: Synthesizes specialist assessments into a final recommendation

Each specialist agent is created with its specific instructions and model configuration.

In [ ]:
# Create specialist agents using the Foundry API
print("🏗️ Creating insurance claims specialist agents...")

# Create the claim validity agent
validity_agent_definition = PromptAgentDefinition(
    model=model_deployment,
    instructions=validity_agent_instructions
)
validity_agent = project_client.agents.create_version(
    agent_name=validity_agent_name,
    definition=validity_agent_definition
)
print(f"✅ Validity agent created: {validity_agent.name} (Version: {validity_agent.version})")

# Create the department assignment agent
department_agent_definition = PromptAgentDefinition(
    model=model_deployment,
    instructions=department_agent_instructions
)
department_agent = project_client.agents.create_version(
    agent_name=department_agent_name,
    definition=department_agent_definition
)
print(f"✅ Department agent created: {department_agent.name} (Version: {department_agent.version})")

# Create the payout estimation agent
payout_agent_definition = PromptAgentDefinition(
    model=model_deployment,
    instructions=payout_agent_instructions
)
payout_agent = project_client.agents.create_version(
    agent_name=payout_agent_name,
    definition=payout_agent_definition
)
print(f"✅ Payout agent created: {payout_agent.name} (Version: {payout_agent.version})")

# Create the orchestrator agent
orchestrator_agent_definition = PromptAgentDefinition(
    model=model_deployment,
    instructions=orchestrator_agent_instructions
)
orchestrator_agent = project_client.agents.create_version(
    agent_name=orchestrator_agent_name,
    definition=orchestrator_agent_definition
)
print(f"✅ Orchestrator agent created: {orchestrator_agent.name} (Version: {orchestrator_agent.version})")

print("\n🎯 All specialist agents created successfully!")

## 📝 Define Workflow YAML

Now we'll create a **declarative YAML workflow** that orchestrates our specialist agents. The workflow:

1. Receives a claim as input
2. Passes the claim through each specialist agent sequentially
3. Collects assessments in variables
4. Sends the combined assessments to the orchestrator for final synthesis

This is the key differentiator - instead of manual Python orchestration, we define the workflow declaratively!

In [ ]:
# Define the workflow YAML that orchestrates our specialist agents
# This YAML defines the sequential flow: Validity -> Department -> Payout -> Orchestrator
# All agents share the same conversation so orchestrator sees full history

workflow_yaml = f"""
kind: workflow
trigger:
  kind: OnConversationStart
  id: claims_processing_workflow
  actions:
    # Store the incoming claim in a variable
    - kind: SetVariable
      id: set_claim_input
      variable: Local.LatestMessage
      value: "=UserMessage(System.LastMessageText)"

    # Step 1: Invoke Validity Agent in MAIN conversation
    - kind: InvokeAzureAgent
      id: validity_assessment
      description: Assess claim validity
      agent:
        name: {validity_agent.name}
      input:
        messages: "=Local.LatestMessage"

    # Step 2: Invoke Department Agent in MAIN conversation
    - kind: InvokeAzureAgent
      id: department_assignment
      description: Assign department
      agent:
        name: {department_agent.name}
      input:
        messages: "=Local.LatestMessage"

    # Step 3: Invoke Payout Agent in MAIN conversation
    - kind: InvokeAzureAgent
      id: payout_estimation
      description: Estimate payout
      agent:
        name: {payout_agent.name}
      input:
        messages: "=Local.LatestMessage"

    # Step 4: Invoke Orchestrator in MAIN conversation
    # Orchestrator sees full conversation history with all specialist assessments
    - kind: InvokeAzureAgent
      id: orchestrator_synthesis
      description: Synthesize all assessments into final report
      agent:
        name: {orchestrator_agent.name}
      input:
        messages:
          - role: user
            content: "Now synthesize all the above assessments into a comprehensive claims report."

    # End the workflow
    - kind: EndConversation
      id: end_workflow
"""

print("📋 Workflow YAML defined successfully!")
print("\n🔄 Workflow Steps:")
print("   1️⃣ Receive claim input")
print("   2️⃣ Invoke Validity Agent → assess claim")
print("   3️⃣ Invoke Department Agent → assign department")
print("   4️⃣ Invoke Payout Agent → estimate payout")
print("   5️⃣ Invoke Orchestrator Agent → synthesize all (sees full conversation)")
print("   6️⃣ End workflow")
print("\n💡 All agents run in the MAIN conversation so orchestrator sees full history")

## 🏗️ Create Workflow Agent

Now we'll create a **WorkflowAgentDefinition** using our YAML workflow. This creates a single "workflow agent" that internally orchestrates all four specialist agents.

In [ ]:
# Create the Workflow Agent using WorkflowAgentDefinition
print("🏗️ Creating workflow agent with YAML definition...")

workflow_agent = project_client.agents.create_version(
    agent_name="claims-processing-workflow",
    definition=WorkflowAgentDefinition(workflow=workflow_yaml),
)

print(f"✅ Workflow agent created!")
print(f"   📛 Name: {workflow_agent.name}")
print(f"   🆔 ID: {workflow_agent.id}")
print(f"   📌 Version: {workflow_agent.version}")

## 📋 Define Sample Insurance Claims

Let's define sample insurance claims to process through our workflow.

In [ ]:
# Sample insurance claims for demonstration
claims = [
    """Claim ID: CLM-2024-001
Policy Type: Auto Insurance
Incident: Vehicle collision at intersection on January 15, 2024.
Description: The insured's vehicle was struck by another car running a red light.
Police report filed. No injuries reported. Vehicle requires bumper replacement and 
alignment repair. Estimated repair cost: $3,200. Deductible: $500.""",
    
    """Claim ID: CLM-2024-002
Policy Type: Home Insurance  
Incident: Water damage from burst pipe on February 3, 2024.
Description: Frozen pipe burst in the basement causing flooding. Damage to flooring,
drywall, and personal belongings. Professional remediation required. 
Estimated damage: $15,000. Policy coverage limit: $250,000.""",
]

print(f"📋 Loaded {len(claims)} sample insurance claims")
for i, claim in enumerate(claims, 1):
    claim_id = claim.split("Claim ID:")[1].split("\n")[0].strip()
    policy_type = claim.split("Policy Type:")[1].split("\n")[0].strip()
    print(f"   • {claim_id} - {policy_type}")

## 🎯 Execute Workflow with Streaming Events

Now we'll run our workflow agent! This demonstrates:
- Creating a conversation for the workflow
- Streaming the workflow execution events
- Watching each agent action as it executes

In [ ]:
# Execute the workflow with streaming for each claim
print(f"🚀 Processing {len(claims)} claims through the workflow agent")
print("=" * 80)

def process_claim_with_workflow(claim_text, claim_number):
    """Process a claim through the workflow agent with streaming events."""
    print(f"\n{'='*80}")
    print(f"📋 PROCESSING CLAIM #{claim_number}")
    print("="*80)
    
    # Create a conversation for this claim
    conversation = openai_client.conversations.create()
    print(f"✅ Created conversation: {conversation.id}")
    
    # Stream the workflow execution
    print("\n🔄 Executing workflow (streaming events)...")
    print("-" * 60)
    
    stream = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": workflow_agent.name, "type": "agent_reference"}},
        input=claim_text,
        stream=True,
    )
    
    final_output = ""
    final_response = None
    all_events = []  # Collect all events for debugging
    
    for event in stream:
        event_type = getattr(event, 'type', None)
        all_events.append(event_type)
        
        # Handle workflow action events
        if event_type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_ADDED:
            if hasattr(event, 'item'):
                item = event.item
                item_type = getattr(item, 'type', None)
                if item_type == ItemType.WORKFLOW_ACTION:
                    action_id = getattr(item, 'action_id', 'unknown')
                    print(f"   ▶️ Action '{action_id}' started...")
        
        elif event_type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_DONE:
            if hasattr(event, 'item'):
                item = event.item
                item_type = getattr(item, 'type', None)
                
                if item_type == ItemType.WORKFLOW_ACTION:
                    action_id = getattr(item, 'action_id', 'unknown')
                    status = getattr(item, 'status', 'completed')
                    print(f"   ✅ Action '{action_id}' completed ({status})")
                    
                    # Check if this action has output/activity text (for SendActivity)
                    if hasattr(item, 'activity') and item.activity:
                        print(f"      📤 Activity output captured")
                        final_output = item.activity
                    if hasattr(item, 'output') and item.output:
                        print(f"      📤 Action output captured")
                        if isinstance(item.output, str):
                            final_output = item.output
                        elif hasattr(item.output, 'text'):
                            final_output = item.output.text
                
                # Capture message content from workflow items
                elif item_type == "message":
                    content = getattr(item, 'content', None)
                    if content:
                        for c in content:
                            if hasattr(c, 'text') and c.text:
                                final_output = c.text
        
        # Capture text output delta
        elif event_type == ResponseStreamEventType.RESPONSE_OUTPUT_TEXT_DELTA:
            if hasattr(event, 'delta') and event.delta:
                final_output += event.delta
        
        # Capture content part events
        elif event_type == ResponseStreamEventType.RESPONSE_CONTENT_PART_ADDED:
            if hasattr(event, 'part'):
                part = event.part
                if hasattr(part, 'text') and part.text:
                    final_output += part.text
        
        elif event_type == ResponseStreamEventType.RESPONSE_CONTENT_PART_DONE:
            if hasattr(event, 'part'):
                part = event.part
                if hasattr(part, 'text') and part.text:
                    if not final_output or part.text not in final_output:
                        final_output = part.text
        
        # Capture completed response
        elif event_type == ResponseStreamEventType.RESPONSE_COMPLETED:
            final_response = getattr(event, 'response', None)
            if final_response:
                # Try to get output_text first
                if hasattr(final_response, 'output_text') and final_response.output_text:
                    final_output = final_response.output_text
                # Also check output items
                elif hasattr(final_response, 'output') and final_response.output:
                    for item in final_response.output:
                        item_type = getattr(item, 'type', None)
                        # Check for activity/message type items
                        if hasattr(item, 'activity') and item.activity:
                            final_output = item.activity
                        content = getattr(item, 'content', None)
                        if content:
                            for c in content:
                                if hasattr(c, 'text') and c.text:
                                    final_output = c.text
    
    print("-" * 60)
    print("\n📊 FINAL CLAIMS REPORT:")
    print("=" * 60)
    
    if final_output:
        print(final_output)
    elif final_response:
        # Debug: Print detailed response information
        print(f"Response received (ID: {getattr(final_response, 'id', 'N/A')})")
        print(f"Status: {getattr(final_response, 'status', 'N/A')}")
        
        # Print all attributes for debugging
        print("\n🔍 DEBUG - Response attributes:")
        for attr in dir(final_response):
            if not attr.startswith('_'):
                val = getattr(final_response, attr, None)
                if val is not None and not callable(val):
                    print(f"   {attr}: {type(val).__name__} = {str(val)[:200]}")
        
        # Print raw output for debugging
        if hasattr(final_response, 'output') and final_response.output:
            print("\n🔍 DEBUG - Workflow Output Items:")
            for i, item in enumerate(final_response.output):
                print(f"  Item {i+1}: type={getattr(item, 'type', 'unknown')}")
                for attr in dir(item):
                    if not attr.startswith('_'):
                        val = getattr(item, attr, None)
                        if val is not None and not callable(val):
                            print(f"    {attr}: {str(val)[:200]}")
    else:
        print("(Workflow completed but no text output captured)")
        print(f"\n🔍 DEBUG - Event types seen: {set(all_events)}")
    
    # Clean up conversation
    openai_client.conversations.delete(conversation_id=conversation.id)
    print(f"\n🧹 Conversation {conversation.id} cleaned up")
    
    return final_output

# Process all claims
all_results = []
for i, claim in enumerate(claims, 1):
    result = process_claim_with_workflow(claim, i)
    all_results.append(result)

print("\n" + "=" * 80)
print("✅ ALL CLAIMS PROCESSED THROUGH WORKFLOW")
print("=" * 80)
print("\n⚠️ DISCLAIMER: This demonstration is for illustrative purposes only.")
print("   Actual insurance claims require review by licensed claims adjusters.")

## 🧹 Clean Up Resources

This cell deletes all the agents we created to avoid leaving resources running in Azure. It's important to clean up agents after use to prevent unnecessary costs.

In [ ]:
# Clean up resources - delete all agents including workflow
print("🧹 Cleaning up all agents...")

# Delete the workflow agent first
project_client.agents.delete_version(agent_name=workflow_agent.name, agent_version=workflow_agent.version)
print(f"✅ Deleted workflow agent: {workflow_agent.name}")

# Delete the specialist agents
project_client.agents.delete_version(agent_name=validity_agent.name, agent_version=validity_agent.version)
print(f"✅ Deleted validity agent: {validity_agent.name}")

project_client.agents.delete_version(agent_name=department_agent.name, agent_version=department_agent.version)
print(f"✅ Deleted department agent: {department_agent.name}")

project_client.agents.delete_version(agent_name=payout_agent.name, agent_version=payout_agent.version)
print(f"✅ Deleted payout agent: {payout_agent.name}")

project_client.agents.delete_version(agent_name=orchestrator_agent.name, agent_version=orchestrator_agent.version)
print(f"✅ Deleted orchestrator agent: {orchestrator_agent.name}")

print("\n🎯 All agents cleaned up successfully!")